<a href="https://colab.research.google.com/github/jmartin1976/BFA_samplescripts/blob/main/CF_GetInfo_from_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Next steps:
###### Look at Bing API
###### retrieve description from url
###### retrieve unstructure available info from K12 https://docs.google.com/spreadsheets/d/1jVZiKFYKh1DRgFMHEyHZkSPA0l2Ssy705QuEoyKr4cg/edit#gid=567602188
###### FUTURE: for JSON to use this, should be fully automated (google sheet extensition?)

### Installation

In [ ]:
# TBD: revise as not all the packages might be finally needed
!pip install langchain
!pip install openai
!pip install chromadb
!pip install Cython
!pip install tiktoken

In [ ]:
!pip install pypdf

In [ ]:
!pip install faiss-cpu

In [ ]:
!apt-get -qq install poppler-utils tesseract-ocr
# Upgrade Pillow to latest version
%pip install -q --user --upgrade pillow
# Install Python Packages
%pip install -q unstructured["local-inference"]==0.8.4

In [6]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### Connect Google Drive

In [7]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [9]:
import os
pdf_folder_path = f'{root_dir}/CF_PDF/'
pdf_files = os.listdir(pdf_folder_path)

### OpenAI API Key

In [10]:
# Get your API keys from openai, you will need to create an account.
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
os.environ["OPENAI_API_KEY"] = ""

### Queries used

In [12]:
# summary_instruction="summarize the text without losing information related to user types, technologies, and founder information"
query='''Please extract the following information: what users are they serving, which technology they use, Stage of the company or current fundraise (pre-see, seed, etc.), what are founder names, what is the gender of each founder member, if the founder has studied in an elite education institution include the institution, if the founder is local or foreign. Provide the answer in this format with one element in each line:
User type:
Technology:
Stage:
Founder name:
Founder Gender:
Elite institution:
Founder local/foreign:
If the information is not available in the dec, write NA'''

### Read some pdfs from a specific folder and extract the requested information

In [13]:
from langchain.document_loaders import UnstructuredFileLoader
from unstructured.partition.pdf import partition_pdf
import openai

for pdf_file in pdf_files:
    print(pdf_file)
    file_path = os.path.join(pdf_folder_path, pdf_file)
    loader = UnstructuredFileLoader(file_path)
    docs = loader.load()
    content=docs[0].page_content[:]
    try:
      # Code to make API request goes here
      response = openai.ChatCompletion.create(
      #model="gpt-3.5-turbo",
      #use 16k version in order to allow processing the docs
      model='gpt-3.5-turbo-16k',
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"{content}\n\n{query}"},
        ]
      )
      print(response['choices'][0]['message']['content'])
    except Exception as e:  # Catch all exceptions
      error_message = str(e)
      if "maximum context length" in error_message:
        print("The request is too long for the model:", e)
      else:
        print("An error ocurred with openAI:", e)


Tolbi - Pitch Deck.pdf
User type: Small scale producers, Agri businesses, Local communities
Technology: Cloud-based AI decision-making platform, Satellite images
Stage: NA
Founder name: Mouhamadou L. KEBE
Founder Gender: Male
Elite institution: NA
Founder local/foreign: Local
Bio-Logical_20230516.pdf
User type: Bio-Logical Nature-based carbon removals for the world's most climate-vulnerable communities.
Technology: Circular economy model, transforming agricultural waste into biochar.
Stage: Not specified.
Founder name: Philip Hunter, Rory Buckworth.
Founder Gender: Not specified.
Elite institution: Not specified.
Founder local/foreign: Not specified.
Thola_20230523.pdf
User type: African farmers, compliance auditors, enterprise buyers
Technology: Compliance automation software, farmer app
Stage: Pre-seed
Founder names: Nneile Nkholise, Dancan Angwenyi, Alan Pohl, Stefan Dominicus, Lalo Ener
Founder Gender: Nneile - female, Dancan - male, Alan - male, Stefan - male, Lalo - NA
Elite inst

### Debug code for specific pdfs

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader
from pypdf import PdfReader
from unstructured.partition.pdf import partition_pdf
from collections import Counter

debug_file='Bio-Logical_20230516.pdf'
file_path = os.path.join(pdf_folder_path, debug_file)
with open(file_path, 'rb') as f:
  pdf_reader = PdfReader(f)
  num_pages = len(pdf_reader.pages)
  raw_text = ''
  for page in range(num_pages):
    page_obj = pdf_reader.pages[page]
    raw_text += page_obj.extract_text()
  print(f'Text from {debug_file}:')
  print(raw_text)
  print(f'Pages with reader {num_pages}')
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
type(pages)
print(f'Pages with pyPDFloader {len(pages)}:')
loader = UnstructuredFileLoader(file_path)
#elements = partition_pdf(file_path)
#display(Counter(type(element) for element in elements))
#print("")
#display(*[(type(element), element.text) for element in elements])

In [70]:
docs = loader.load()

In [77]:
type(docs)

list

In [88]:
docs[0].page_content[:]

"CREATING GROWTH FROM WASTE\n\nBio-Logical Nature-based carbon removals for the world's most climate-vulnerable communities.\n\nBio-Logical is a UK registered company number 14227765\n\nWHO WHO WE ARE. WE ARE.\n\nBio-Logical fights climate change and supports the world’s most climate-vulnerable communities through the power of nature-based carbon removals. By 2030 we aim to sequester 1 million tons of CO2 and support 1 million smallholder farmers, helping them to regenerate their land, grow more food and improve their livelihoods.\n\nThe Problem Over the last five years, African nations have been, and will continue to be, some of the hardest hit by climate change. African smallholders are already experiencing 1.5x the global mean temperature rise, severe droughts, and soil degradation, which are causing widespread crop failure. Whilst it is positive that the world is turning its attention towards fighting climate change, the impacts felt today in Africa are not receiving the attention 

In [ ]:
docs[0].page_content[:]

In [92]:
content=docs[0].page_content[:]
try:
# Code to make API request goes here
  response = openai.ChatCompletion.create(
  #model="gpt-3.5-turbo",
  model='gpt-3.5-turbo-16k',
  messages=[
   {"role": "system", "content": "You are a helpful assistant."},
   {"role": "user", "content": f"{content}\n\n{query}"},
  ]
  )
  print(response['choices'][0]['message']['content'])
except Exception as e:  # Catch all exceptions
  error_message = str(e)
  if "maximum context length" in error_message:
    print("The request is too long for the model:", e)

  else:
    print("An error ocurred with openAI:", e)


User type: Bio-Logical serves both smallholder farmers in climate-vulnerable communities in Africa and companies and nations looking to offset emissions through nature-based carbon removals.

Technology: Bio-Logical uses a circular economy model to transform agricultural waste into biochar, a super material that sequesters carbon, regenerates degraded soil, and improves crop resilience.

Founder names: The founders of Bio-Logical are Philip Hunter and Rory Buckworth.

Founder gender: The gender of the founders is not specified in the provided information.
